# Prepare Chicago Taxi Trips dataset

We will use data from the public [Chicago Taxi Trips](https://console.cloud.google.com/marketplace/product/city-of-chicago-public-data/chicago-taxi-trips) dataset 
* includes taxi trips from 2013 to the present, reported to the City of Chicago in its role as a regulatory agency. 
* To protect the privacy of the drivers and users of the cab at the same time and allow the aggregator to analyze the data, the Taxi ID is kept consistent for any given taxi medallion number but does not show the number, Census Tracts are suppressed in some cases, and times are rounded to the nearest 15 minutes.

> For more information, check out [Chicago Taxi Trips on Marketplace](https://console.cloud.google.com/marketplace/product/city-of-chicago-public-data/chicago-taxi-trips)

### imports 

In [ ]:
from google.cloud import bigquery
from google.cloud import storage

print(f'bigquery SDK version: {bigquery.__version__}')

In [ ]:
PROJECT_ID = "hybrid-vertex"
REGION = "us-central1"

# Set the project id
! gcloud config set project {PROJECT_ID}

In [ ]:
bq_client = bigquery.Client(
    project=PROJECT_ID,
    location=REGION
)

## Create BQ dataset

In [ ]:
BQ_DATASET_NAME = "mlops"
bq_client.query(f'CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.{BQ_DATASET_NAME}`').result()

## Create BQ table

In [ ]:
QUERY = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.mlops.chicago`
AS (
    WITH
      taxitrips AS (
      SELECT
        trip_start_timestamp,
        trip_end_timestamp,
        trip_seconds,
        trip_miles,
        payment_type,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        tips,
        tolls,
        fare,
        pickup_community_area,
        dropoff_community_area,
        company,
        unique_key
      FROM
        `{PROJECT_ID}.mlops.taxi_trips`
      WHERE pickup_longitude IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND trip_miles > 0
      AND trip_seconds > 0
      AND fare > 0
      AND EXTRACT(YEAR FROM trip_start_timestamp) = 2019
    )

    SELECT
      trip_start_timestamp,
      EXTRACT(MONTH from trip_start_timestamp) as trip_month,
      EXTRACT(DAY from trip_start_timestamp) as trip_day,
      EXTRACT(DAYOFWEEK from trip_start_timestamp) as trip_day_of_week,
      EXTRACT(HOUR from trip_start_timestamp) as trip_hour,
      trip_seconds,
      trip_miles,
      payment_type,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(pickup_longitude, pickup_latitude), 0.1)
      ) AS pickup_grid,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0.1)
      ) AS dropoff_grid,
      ST_Distance(
          ST_GeogPoint(pickup_longitude, pickup_latitude),
          ST_GeogPoint(dropoff_longitude, dropoff_latitude)
      ) AS euclidean,
      CONCAT(
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickup_longitude,
              pickup_latitude), 0.1)),
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropoff_longitude,
              dropoff_latitude), 0.1))
      ) AS loc_cross,
      IF((tips/fare >= 0.2), 1, 0) AS tip_bin,
      tips,
      tolls,
      fare,
      pickup_longitude,
      pickup_latitude,
      dropoff_longitude,
      dropoff_latitude,
      pickup_community_area,
      dropoff_community_area,
      company,
      unique_key,
      trip_end_timestamp
    FROM
      taxitrips
    LIMIT 1000000
)
"""
# print to inspect
print(QUERY)

In [ ]:
# # uncomment to submit
job = bq_client.query(query = QUERY)
job.result()
(job.ended-job.started).total_seconds()